# インポート

In [1]:
# import packages

# python
import os
from glob import glob
import math
from typing import Dict
import sys

# IPython
from IPython.display import display

# numpy
import numpy as np

# scipy
import scipy.signal
from scipy.optimize import curve_fit
from scipy import integrate

# sympy
import sympy

# bokeh
from bokeh.plotting import output_notebook, figure, show
output_notebook()

# local
from WavData import WavData
import FileUtil
import SignalProcessingUtil as spu
import NotebookUtil as nu

Loading BokehJS ...

Loading BokehJS ...

# データディレクトリ内のファイルを列挙

In [2]:
# enumerate
top_dir = os.environ.get('PWD')
display(f'top_dir={top_dir}')
wav_files = glob(os.path.join(top_dir, 'data/*.wav'))

# dump
wav_files

'top_dir=/kick_extractor'

['/kick_extractor/data/blue_eyes.wav',
 '/kick_extractor/data/Freaking Tight - Alex Prospect_2.wav',
 '/kick_extractor/data/last_goodbye_mob.wav',
 '/kick_extractor/data/WDYWFM - Alex Prospect_2.wav_est.wav_est.wav',
 '/kick_extractor/data/SHOT ME DOWN - Alex Prospect 2_2.wav',
 '/kick_extractor/data/Tremor - Alex Prospect & WILSXN_2.wav',
 '/kick_extractor/data/02 Dreamer (7iva & saqwz Remix).wav',
 '/kick_extractor/data/satellite.wav',
 '/kick_extractor/data/happy_days_refrain_20180421_2.wav',
 '/kick_extractor/data/save_a_life.wav',
 '/kick_extractor/data/Heaven 2017 - Alex Prospect_2.wav',
 '/kick_extractor/data/stay_young.wav',
 '/kick_extractor/data/Never Forget You - Alex Prospect_2.wav',
 '/kick_extractor/data/WDYWFM - Alex Prospect_2.wav_est.wav',
 '/kick_extractor/data/look_back.wav',
 '/kick_extractor/data/WDYWFM - Alex Prospect_2.wav',
 '/kick_extractor/data/Need U 100% - Alex Prospect & Spyro_2.wav',
 '/kick_extractor/data/all_about_elysium.wav',
 '/kick_extractor/data/INT

# 入力ファイルをロード

In [3]:
# TODO 決め打ち
#src_path = '/kick_extractor/data/WDYWFM - Alex Prospect_2.wav_est.wav'
src_path = '/kick_extractor/data/WDYWFM - Alex Prospect_2.wav'

# load
wavdata_raw = FileUtil.load_wav_file(src_path)

# plot
nu.plot( nu.describe_wavdata('original waveform', wavdata_raw) )

/kick_extractor/source/FileUtil.py:45: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, samples = wavfile.read(src_path)


# pre-process wav data

In [4]:
# TODO 決め打ちのパラメータ
beat_per_minute = 170.0
beat_length_in_sec = 60.0 / 170.0
peak_filter_initial_frequency_in_hz = 80.0
peak_filter_bandwidth_in_hz = 100.0 #200.0
upsampling_rate = 1

# モノラルにミックスダウン
if wavdata_raw.samples.ndim != 1:
    wavdata_mono = WavData(wavdata_raw.sample_rate, np.mean(wavdata_raw.samples, axis=0))
else:
    wavdata_mono = wavdata_raw
    
# 先頭 1/8 音符分を切り出し
beat_length_in_sample = beat_length_in_sec * wavdata_raw.sample_rate
wavdata_truncated = WavData(wavdata_mono.sample_rate, wavdata_mono.samples[:int(beat_length_in_sample/2)])

# 最も大きい正弦波以外の周波数帯をカットオフして「キレイ」にする
wavdata_smoothed_max = np.zeros_like(wavdata_truncated.samples)
wavdata_smoothed_min = np.zeros_like(wavdata_truncated.samples)
for f in [f for f in np.geomspace(40, 240, 256)]:
    for q in [q for q in np.linspace(100, 200, 16)]:
        wavdata_filtered = spu.apply_peak_filter(wavdata_truncated, peak_filter_bandwidth_in_hz, f).samples
        wavdata_smoothed_max = np.fmax(wavdata_smoothed_max, wavdata_filtered)
        wavdata_smoothed_min = np.fmin(wavdata_smoothed_min, wavdata_filtered)
wavdata_smoothed = WavData(wavdata_truncated.sample_rate, wavdata_smoothed_max + wavdata_smoothed_min)

# アップサンプリング
samples_prepro = scipy.signal.resample(wavdata_smoothed.samples, wavdata_smoothed.length_in_samples()*upsampling_rate)
wavdata_prepro = WavData(wavdata_smoothed.sample_rate*upsampling_rate, samples_prepro)

# 結果をプロット
nu.plot(
    nu.describe_wavdata('original waveform', wavdata_truncated) +
    nu.describe_wavdata('smoothed waveform', wavdata_smoothed) +
    nu.describe_wavdata('pre-processed', wavdata_prepro)
)

# 極値を探索

In [5]:
# 結合
min_positions_in_samples = np.asarray(scipy.signal.argrelmin(wavdata_prepro.samples, order=4)).flatten()
max_positions_in_samples = np.asarray(scipy.signal.argrelmax(wavdata_prepro.samples, order=4)).flatten()
raw_extrema_positions_in_sample = np.sort(np.concatenate([min_positions_in_samples,max_positions_in_samples]).flatten())
display(type(raw_extrema_positions_in_sample))

# プロット
nu.plot(
    nu.describe_dot_on_wavdata('extrema', wavdata_prepro, raw_extrema_positions_in_sample) +
    nu.describe_wavdata('wavdata_prepro', wavdata_prepro)
)

numpy.ndarray

# 隣接＆同符号の極値をグループとみなして代表点で置き換える
- 以下の条件を両方とも満たす極値でグループを作る
    - 隣接している
    - 符号が同一である
- グループごとに代表点を算出して、オリジナルの極値を置き換える
    - 振幅を重みとしたサンプル位置の平均＝代表点

In [6]:
# for ループで頑張る
extrema_positions_in_sample = []
position_buffer = []
value_buffer = []
for position in raw_extrema_positions_in_sample:
    value = wavdata_prepro.samples[position]
    sign = value > 0.0
    if len(position_buffer) != 0:
        last_position = position_buffer[-1]
        last_value = value_buffer[-1]
        last_sign = last_value > 0.0
        if sign != last_sign:
            averaged_position = np.average(position_buffer, weights=np.abs(value_buffer))
            extrema_positions_in_sample.append(averaged_position)
            position_buffer.clear()
            value_buffer.clear()
    position_buffer.append(position)
    value_buffer.append(value)
extrema_positions_in_sample = np.asarray(extrema_positions_in_sample).astype(np.int32)
    
# プロット
nu.plot(
    nu.describe_dot_on_wavdata('extrema', wavdata_prepro, extrema_positions_in_sample) +
    nu.describe_wavdata('wavdata_prepro', wavdata_prepro)
)

# 極端な外れ値を除外
- 外れ値リストの中央を始点として外れ値を探索する
- 極値の極性が隣接極値と反対になってない＝反対になってない隣接極値を外れ値とみなす
- 外れ値で挟まれた閉区間の外側を外れ値として除外する

- 必要なくなったので **バイパス** している
- 隣接同符号のグルーピングのおかげでこの処理が不要になった

In [7]:
'''
# 「どちらか一方の隣接極値と符号が同一なら True 」な配列を生成(is_outlier)
extrema_signes = wavdata_prepro.samples[extrema_positions_in_sample] > 0.0
extrema_signes_shift_left = np.roll(extrema_signes, -1)
extrema_signes_shift_left[-1] = extrema_signes[-1]
extrema_signes_shift_right = np.roll(extrema_signes, 1)
extrema_signes_shift_right[0] = extrema_signes[0]
is_outlier = ( extrema_signes == extrema_signes_shift_left ) | ( extrema_signes == extrema_signes_shift_right )

# 中央から左（過去）方向に向けて外れ値を探索
initial_extrema_index = int(len(extrema_positions_in_sample) / 2)
outlier_index_left = initial_extrema_index
outlier_index_right = initial_extrema_index
while ( 0 < outlier_index_left ) and ( not is_outlier[outlier_index_left] ):
    outlier_index_left = outlier_index_left - 1
while ( outlier_index_right < is_outlier.size - 1 ) and ( not is_outlier[outlier_index_right] ):
    outlier_index_right = outlier_index_right + 1

# 切り出し
inlier_extrema_positions_in_samples = extrema_positions_in_sample[outlier_index_left:outlier_index_right]
'''
inlier_extrema_positions_in_samples = extrema_positions_in_sample

# プロット
nu.plot(
    nu.describe_dot_on_wavdata('extrema', wavdata_prepro, inlier_extrema_positions_in_samples) +
    nu.describe_wavdata('wavdata_prepro', wavdata_prepro)
)

# ゼロクロス点（サブサンプル精度）を抽出
- ゼロをまたぐ２点に直線を当てはめて

In [8]:
# 全てのゼロクロス点を抽出
zero_cross_point_in_samples = np.asarray( np.nonzero( ( wavdata_prepro.samples[:-1] > 0.0 ) ^ ( np.roll(wavdata_prepro.samples, -1)[:-1] > 0.0 ) ) ).flatten()

# インライア近傍のゼロクロス点を抽出
inliner_zero_cross_point_in_samples = zero_cross_point_in_samples[(inlier_extrema_positions_in_samples[0] < zero_cross_point_in_samples) & (zero_cross_point_in_samples < inlier_extrema_positions_in_samples[-1])]

# サブサンプルオフセットを推定
#
# ゼロクロス点前後のサンプルを通る直線は
# y = (samples[i+1] - samples[i]) * x + samples[i]
# これを y = 0 として x について解くと
# x = -samples[i] / (samples[i+1] - samples[i])
# これをベクトル演算で頑張ると↓になる
slope = wavdata_prepro.samples[inliner_zero_cross_point_in_samples+1] - wavdata_prepro.samples[inliner_zero_cross_point_in_samples]
intercept = wavdata_prepro.samples[inliner_zero_cross_point_in_samples]
subsample_offset = -intercept / slope

# サブサンプル精度のゼロクロス点を計算
inliner_zero_cross_points_in_subsamples = inliner_zero_cross_point_in_samples + subsample_offset

# プロット
nu.plot(
    nu.describe_dot_on_wavdata('zero-cross', wavdata_prepro, inliner_zero_cross_points_in_subsamples) +
    nu.describe_wavdata('wavdata_prepro', wavdata_prepro)
)

# 極値位置をサブサンプル精度化
- トゥルーピーク（インターサンプルピーク）を得る
- 極値と隣接するサンプルの合計３点でいい感じに重み付き平均みたいなことをする

In [9]:
def extrema_position_to_subsample(samples: np.ndarray, position_in_samples: int):
    '''
    position_in_samples をサブサンプル精度化して返却する。
    '''
    # エイリアス
    magnitude_left = abs(samples[position_in_samples-1])
    magnitude_center = abs(samples[position_in_samples])
    magnitude_right = abs(samples[position_in_samples+1])
    magnitude_1st = magnitude_center
    magnitude_2nd = max(magnitude_left, magnitude_right)
    magnitude_3rd = min(magnitude_left, magnitude_right)
    # サブサンプル単位のオフセットを計算
    subsample_offset = 0.5 * (magnitude_2nd - magnitude_3rd) / (magnitude_1st - magnitude_3rd)
    if magnitude_left > magnitude_right:
        return position_in_samples - subsample_offset
    else:
        return position_in_samples + subsample_offset

# サブサンプル精度化
inlier_extrema_positions_in_subsamples = np.array([extrema_position_to_subsample(wavdata_prepro.samples, position_in_samples) for position_in_samples in inlier_extrema_positions_in_samples])

# プロット
nu.plot(
    nu.describe_dot_on_wavdata('extrema', wavdata_prepro, inlier_extrema_positions_in_samples) +
    nu.describe_dot_on_wavdata('extrema(sub)', wavdata_prepro, inlier_extrema_positions_in_subsamples) +
    nu.describe_wavdata('wavdata_prepro', wavdata_prepro)
)

# EDA 用にゼロクロスと極値をマージ
- 使ってないので **バイパス**

In [10]:
# DEBUG
inlier_control_points_in_subsamples = np.sort( np.concatenate([inlier_extrema_positions_in_subsamples, inliner_zero_cross_points_in_subsamples]) )

# プロット
nu.plot(
    nu.describe_dot_on_wavdata('control point', wavdata_prepro, inlier_control_points_in_subsamples) +
    nu.describe_wavdata('wavdata_prepro', wavdata_prepro),
    beat_per_minute=beat_per_minute
)

# ゼロクロスと極値で比較
- ゼロクロス点と極値を合成してから周波数エンベロープを算出するのはすごく微妙

In [11]:
nu.plot(
    nu.describe_frequency('extrema(subsamples)', inlier_extrema_positions_in_subsamples, wavdata_prepro.sample_rate, 2) +
    nu.describe_frequency('zero-cross(subsamples)', inliner_zero_cross_points_in_subsamples, wavdata_prepro.sample_rate, 2) +
    nu.describe_frequency('ctrl(subsamples)', inlier_control_points_in_subsamples, wavdata_prepro.sample_rate, 4),
    is_log_scale=True
)
nu.plot(
    nu.describe_frequency('extrema(subsamples)', inlier_extrema_positions_in_subsamples, wavdata_prepro.sample_rate, 2) +
    nu.describe_frequency('zero-cross(subsamples)', inliner_zero_cross_points_in_subsamples, wavdata_prepro.sample_rate, 2),
    is_log_scale=True
)
nu.plot(
    nu.describe_frequency('ctrl(subsamples)', inlier_control_points_in_subsamples, wavdata_prepro.sample_rate, 4),
    is_log_scale=True
)

# ゼロクロスベース・極値ベースの周波数遷移を合成
- やりかた
    - 極値リストとゼロクロス点リストとでそれぞれ周波数エンベロープリストを計算する
    - その後、周波数エンベロープリストをマージする
- 理由
    - 制限波の山が微妙にスキューしている関係で極値のが全体的にシフトしている
    - そのため、ゼロクロス点と混ぜる存在しない疎密が発生する
    - この疎密が原因で周波数エンベロープが劣化してしまう
    - なので、独立に周波数エンベロープを算出して結果をマージする

In [12]:
# 単純に
inlier_control_points_in_hz = np.concatenate([
    spu.to_frequency(inlier_extrema_positions_in_subsamples, wavdata_prepro.sample_rate, 2),
    spu.to_frequency(inliner_zero_cross_points_in_subsamples, wavdata_prepro.sample_rate, 2)
])
inlier_control_points_in_subsamples = np.concatenate([inlier_extrema_positions_in_subsamples[:-1], inliner_zero_cross_points_in_subsamples[:-1]])

# 並び順を修正
sort_indices = np.argsort(inlier_control_points_in_subsamples)
inlier_control_points_in_hz = inlier_control_points_in_hz[sort_indices]
inlier_control_points_in_subsamples = inlier_control_points_in_subsamples[sort_indices]

# 位置情報を秒単位に修正
inlier_control_points_in_sec = inlier_control_points_in_subsamples / wavdata_prepro.sample_rate

# プロット
nu.plot(
    nu.describe_scatter('frquency(Hz)', inlier_control_points_in_hz, inlier_control_points_in_sec),
    is_log_scale=True,
    beat_per_minute=beat_per_minute
)

# キックの数式モデルを定義
- 以下の２パターン
    - 秒 --> Hz
    - 秒 --> 位相（無単位）

In [13]:
# シンボルを定義
sym_x = sympy.Symbol('x')
sym_exp_base = sympy.Symbol('exp_base')
sym_x_scaler = sympy.Symbol('x_scaler')
sym_x_offset = sympy.Symbol('x_offset')
sym_y_scaler = sympy.Symbol('y_scaler')
sym_y_offset = sympy.Symbol('y_offset')
sym_phase_offset = sympy.Symbol('phase_offset')

# 式を定義
exp_2freq = sym_y_scaler * sym_exp_base ** (sym_x_scaler * sym_x + sym_x_offset) + sym_y_offset
exp_2phase = sympy.integrate(exp_2freq, sym_x)
exp_2phase_def = exp_2phase + sym_phase_offset

# 式を TeX レンダリング
display(exp_2freq)
display(exp_2phase)
display(exp_2phase_def)

# sympy 式 --> numpy 式
sym_args = (sym_x, sym_exp_base, sym_x_scaler, sym_x_offset, sym_y_scaler, sym_y_offset)
sym_args_def = (sym_x, sym_exp_base, sym_x_scaler, sym_x_offset, sym_y_scaler, sym_y_offset, sym_phase_offset)
target_function_2freq = sympy.lambdify(sym_args, exp_2freq, 'numpy')
target_function_2phase = sympy.lambdify(sym_args, exp_2phase, 'numpy')
target_function_2phase_def = sympy.lambdify(sym_args_def, exp_2phase_def, 'numpy')

exp_base**(x*x_scaler + x_offset)*y_scaler + y_offset

x*y_offset + Piecewise((exp_base**(x*x_scaler + x_offset)*y_scaler/(x_scaler*log(exp_base)), Ne(x_scaler*log(exp_base), 0)), (x*y_scaler, True))

phase_offset + x*y_offset + Piecewise((exp_base**(x*x_scaler + x_offset)*y_scaler/(x_scaler*log(exp_base)), Ne(x_scaler*log(exp_base), 0)), (x*y_scaler, True))

# 周波数エンベロープを当てはめ
- 周波数の推移データに対して何らかの数式的なモデルを当てはめる
- それによりモデルのパラメータを得る
- 外れ値に対する頑健性がほしいので LMeds を使う

In [14]:
# 層化ランダム選択（インデックス返却）
def stratrified_random_choice(stop, div):
    '''
    [0, stop) の範囲を等間隔な div 個の区間に分割し、
    区間ごとに１つの要素をランダム選択する。
    '''
    regions = np.linspace(0, stop, div+1).astype(np.int32)
    regions_size = np.roll(regions, -1)[:-1] - regions[:-1]
    return (regions[:-1] + regions_size * np.random.rand(regions_size.size)).astype(np.int32)

# LMedS でフィッティング
def lmeds(target_function, array_x: np.ndarray, array_y: np.ndarray, initial_params: list, num_iteration: int = 1000):
    np.random.seed(seed=20220116)
    fitting_param = None
    fitting_param_eval = None
    for i in range(num_iteration):
        random_indices = stratrified_random_choice(array_x.size, int(array_x.size/2))
        try:
            temp_fitting_param, _ = curve_fit(
                target_function,
                array_x[random_indices],
                array_y[random_indices],
                p0=initial_params
            )
        except Excpection as e:
            Display(e)
            continue
        temp_residual = target_function(array_x, *temp_fitting_param) - array_y
        temp_fitting_param_eval = np.sort(temp_residual ** 2)[int(array_x.size/2)]
        if fitting_param_eval is None or temp_fitting_param_eval < fitting_param_eval:
            fitting_param = temp_fitting_param
            fitting_param_eval = temp_fitting_param_eval
    return fitting_param
            
# LMedS でフィッティング
freq_fitting_param = lmeds(
    target_function_2freq,
    inlier_control_points_in_sec,
    inlier_control_points_in_hz,
    [1.0, -1.0, 0.0, 1.0, 0.0]
)

# フィッティングのパラメータを対象関数にバインド
estimated_function_offset2frequency = lambda x: target_function_2freq(x, *freq_fitting_param)
estimated_function_offset2phase = lambda x: target_function_2phase(x, *freq_fitting_param)

# フィッティングの結果をプロット
fitting_points_in_sec = inlier_control_points_in_sec
fitting_points_in_hz = estimated_function_offset2frequency(fitting_points_in_sec)
nu.plot(
    nu.describe_scatter('source(hz)', inlier_control_points_in_hz, inlier_control_points_in_sec)+
    nu.describe_scatter('fitting(hz)', fitting_points_in_hz, fitting_points_in_sec),
    is_log_scale=True,
    beat_per_minute=beat_per_minute
)

<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in power
  return exp_base**(x*x_scaler + x_offset)*y_scaler + y_offset
/usr/local/lib/python3.8/dist-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


# 週蓮エンベロープ当てはめ結果を元に波形を再構築
- モデルと推定したパラメータに基づいて正弦波を再生

In [15]:
# 推定した周波数関数を元に波形を再生
estimated_positions_in_samples = np.arange(0, beat_length_in_sample/2)
estimated_positions_in_sec = estimated_positions_in_samples / wavdata_prepro.sample_rate
estimated_phases_in_unit = estimated_function_offset2phase(estimated_positions_in_sec)
estimated_samples = np.sin(estimated_phases_in_unit * 2.0 * np.pi)

# プロット
estimated_wavdata = WavData(wavdata_prepro.sample_rate, estimated_samples)
nu.plot(
    nu.describe_wavdata('original waveform', wavdata_prepro) +
    nu.describe_wavdata('estimated waveform', estimated_wavdata)
)

# 位相エンベロープで当てはめを行う

In [16]:
# 極性 --> 位相
def polarity_to_phase_sub(prev_p, p, next_p, period):
    if p == 1:
        return 0.25 + period, period
    elif p == -1:
        return 0.75 + period, period + 1
    elif prev_p is not None:
        if prev_p == 1:
            return 0.50 + period, period
        else:
            return 0.00 + period, period
    elif next_p is not None:
        if next_p == 1:
            return 0.00 + period, period
        else:
            return 0.50 + period, period
    else:
        return None, period

# 位相リストを作成
def polarity_to_phase(wavdata: WavData, positions_in_samples: np.ndarray):
    # TODO 先頭要素の位相を確定させたらパターンをフィルで良いと思う
    magnitudes_in_unit = wavdata.samples[inlier_control_points_in_subsamples.astype(np.int16)]
    polarities = np.asarray([0 if abs(m) < 0.1 else 1 if m >0 else -1 for m in magnitudes_in_unit])
    phases_in_unit = []
    period = 0
    for i in range(polarities.size):
        if i == 0:
            phase, period = polarity_to_phase_sub(
                None,
                polarities[i],
                polarities[i+1],
                period
            )
        elif i==(polarities.size-1):
            phase, period = polarity_to_phase_sub(
                polarities[i-1],
                polarities[i],
                None,
                period
            )
        else:
            phase, period = polarity_to_phase_sub(
                polarities[i-1],
                polarities[i],
                polarities[i+1],
                period
            )
        phases_in_unit.append(phase)
    return np.asarray(phases_in_unit)

# 位相リストを生成
target_phases_in_unit = polarity_to_phase(wavdata_prepro, inlier_control_points_in_subsamples)

# プロット
nu.plot(
    nu.describe_scatter('source', target_phases_in_unit, inlier_control_points_in_sec)+
    nu.describe_scatter('estimated', estimated_phases_in_unit, estimated_positions_in_sec),
    is_log_scale=False,
    beat_per_minute=beat_per_minute
)

# LMedS でフィッティング
phase_fitting_param = lmeds(
    target_function_2phase_def,
    inlier_control_points_in_sec,
    target_phases_in_unit,
    [10.0, -10.0, 0.0, 10.0, 0.0, 0.0],#np.concatenate([freq_fitting_param,np.asarray([0.0])]),
    100
)

display(f'phase_fitting_param = {phase_fitting_param}')

# フィッティングのパラメータを対象関数にバインド
estimated_function_2phase_def = lambda x: target_function_2phase_def(x, *phase_fitting_param)

# 推定した周波数関数を元に波形を再生
estimated_positions_in_samples = np.arange(0, beat_length_in_sample/2)
estimated_positions_in_sec = estimated_positions_in_samples / wavdata_prepro.sample_rate
estimated_phases_in_unit = estimated_function_2phase_def(estimated_positions_in_sec)
estimated_samples = np.sin(estimated_phases_in_unit * 2.0 * np.pi)

# フィッティングの結果をプロット
estimated_wavdata = WavData(wavdata_prepro.sample_rate, estimated_samples)
nu.plot(
    nu.describe_wavdata('original waveform', wavdata_prepro) +
    nu.describe_wavdata('estimated waveform', estimated_wavdata)
)

<lambdifygenerated-3>:2: RuntimeWarning: invalid value encountered in log
  return phase_offset + x*y_offset + select([not_equal(x_scaler*log(exp_base), 0),True], [exp_base**(x*x_scaler + x_offset)*y_scaler/(x_scaler*log(exp_base)),x*y_scaler], default=nan)
<lambdifygenerated-3>:2: RuntimeWarning: invalid value encountered in power
  return phase_offset + x*y_offset + select([not_equal(x_scaler*log(exp_base), 0),True], [exp_base**(x*x_scaler + x_offset)*y_scaler/(x_scaler*log(exp_base)),x*y_scaler], default=nan)
<lambdifygenerated-3>:2: RuntimeWarning: overflow encountered in power
  return phase_offset + x*y_offset + select([not_equal(x_scaler*log(exp_base), 0),True], [exp_base**(x*x_scaler + x_offset)*y_scaler/(x_scaler*log(exp_base)),x*y_scaler], default=nan)
<lambdifygenerated-3>:2: RuntimeWarning: overflow encountered in multiply
  return phase_offset + x*y_offset + select([not_equal(x_scaler*log(exp_base), 0),True], [exp_base**(x*x_scaler + x_offset)*y_scaler/(x_scaler*log(exp_ba

'phase_fitting_param = [  5.83283675 -16.6646199    1.01451591  40.11268233  49.79607822\n   5.99497984]'

# 終了位相を調整

# 再構築波形をファイル出力

In [17]:
dst_path = src_path + '_est.wav'
FileUtil.save_wav_file(dst_path, estimated_wavdata)

# やってみたけど、結局要らなくなったことリスト

## 検出した極値が正負交互に来る範囲のくくりだし
- 中央らへんからスタートして「正 --> 負 --> 正 --> 負 --> ...」っていう規則が守られている範囲を広げていく
- 「隣接する同符号の極値をグループ化して代表点で置き換える」処理を適用した後なら「正負ルール」が必ず守られているので不要になった